In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, classification_report
from sklearn.impute import SimpleImputer

**Description:** 

The script performs predictive modeling to explore how cardiovascular surgical interventions relate to national cardiovascular death rates.
Independent variables (X): Rates of surgical procedures (Bypass_2018, Bypass_change, Angioplasty_2018, Angioplasty_change).
Dependent variable (y): Standardized Death Rate (SDR_total).
Intuition: Countries with higher/lower use of certain interventions may experience differences in cardiovascular mortality.

* Step 1: Loading datasets
* Step 2: Feature formation (features: Bypass_2018, Bypass_change, Angioplasty_2018, Angioplasty_change) 
* Step 3: Removing the data of the countries which are not present in both datasets
* Step 4: Imputing missing features by the mean data
* Step 5: Performing linear regression
* Step 6: Performing random forest regression
* Step 7: Binarization as high and low
* Step 8: Performing a logistic regression
* Step 9: Performing random forest classifier

In [2]:
df_deaths = pd.read_csv("data/processed/deaths_clean.csv")
df_surg   = pd.read_csv("data/processed/cvd_surgery_clean.csv")


df_surg["Bypass_change"] = df_surg["Bypass_2023"] - df_surg["Bypass_2018"]
df_surg["Angioplasty_change"] = df_surg["Angioplasty_2023"] - df_surg["Angioplasty_2018"]


common_countries = df_surg["Country"].isin(df_deaths["Country"])
df_surg = df_surg[common_countries]
df_deaths = df_deaths[df_deaths["Country"].isin(df_surg["Country"])]


X = df_surg.set_index("Country")[["Bypass_2018", "Angioplasty_2018",
                                  "Bypass_change", "Angioplasty_change"]]
y_reg = df_deaths.set_index("Country").loc[X.index, "SDR_total"]

#
mask = y_reg.notna()
X = X.loc[mask]
y_reg = y_reg.loc[mask]


imputer = SimpleImputer(strategy="mean")
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)


print("\n--- REGRESSION ---\n")

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_reg, test_size=0.2, random_state=42)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lr = lin_reg.predict(X_test)

print("Linear Regression")
print("MSE:", round(mean_squared_error(y_test, y_pred_lr),2))
print("R2:", round(r2_score(y_test, y_pred_lr),2))

rf_reg = RandomForestRegressor(n_estimators=200, random_state=42)
rf_reg.fit(X_train, y_train)
y_pred_rf = rf_reg.predict(X_test)

print("\nRandom Forest Regression")
print("MSE:", round(mean_squared_error(y_test, y_pred_rf),2))
print("R2:", round(r2_score(y_test, y_pred_rf),2))
print("Feature importance:")
print(pd.Series(rf_reg.feature_importances_, index=X_imputed.columns).sort_values(ascending=False))


print("\n--- CLASSIFICATION ---\n")


median_sdr = y_reg.median()
y_class = (y_reg > median_sdr).astype(int)


X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y_class, test_size=0.2, random_state=42, stratify=y_class
)


lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

print("Logistic Regression")
print("Accuracy:", round(accuracy_score(y_test, y_pred_lr),2))
print("F1 score:", round(f1_score(y_test, y_pred_lr),2))
print(classification_report(y_test, y_pred_lr))


rf_model = RandomForestClassifier(n_estimators=300, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Random Forest")
print("Accuracy:", round(accuracy_score(y_test, y_pred_rf),2))
print("F1 score:", round(f1_score(y_test, y_pred_rf),2))
print(classification_report(y_test, y_pred_rf))
print("Feature importance:")
print(pd.Series(rf_model.feature_importances_, index=X_imputed.columns).sort_values(ascending=False))

print("\n====================================================")
print("Regression struggles with noisy SDR values.")
print("Classification (Low/High SDR) performs more robustly.")
print("====================================================\n")



--- REGRESSION ---

Linear Regression
MSE: 66753.2
R2: -0.47

Random Forest Regression
MSE: 47956.44
R2: -0.05
Feature importance:
Angioplasty_change    0.488093
Angioplasty_2018      0.181938
Bypass_change         0.170038
Bypass_2018           0.159931
dtype: float64

--- CLASSIFICATION ---

Logistic Regression
Accuracy: 0.71
F1 score: 0.67
              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.67      0.67      0.67         3

    accuracy                           0.71         7
   macro avg       0.71      0.71      0.71         7
weighted avg       0.71      0.71      0.71         7

Random Forest
Accuracy: 0.86
F1 score: 0.86
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.75      1.00      0.86         3

    accuracy                           0.86         7
   macro avg       0.88      0.88      0.86         7
weighted av

**Interpretation:**

Regression (Linear & Random Forest): Poor predictive performance (low R², high MSE), may be due to:
- Small dataset (few countries).
- Noisy target variable (death rates influenced by many factors beyond surgeries).

2-class classification (Low vs High SDR) performs successfully. :
Logistic Regression: Accuracy ~71%, F1 ~67%.
Random Forest: Accuracy & F1 ~86% is surprisingly strong given the small sample.

Angioplasty_change and Bypass_2018 appear to be the strongest predictors of whether a country falls in the High vs Low SDR group.

Thr reason of regression failing may be because predicting exact death rates from only 4 procedure variables is too noisy.
Classification works better because it reduces the problem to a simpler “High/Low” categorization.